In [6]:
using Pkg; Pkg.activate(".")

  Activating project at `D:\Documents\Documents Bertrand\_Poly\_H2022\MTH8408 Optimisation\Projet Optimisation git`


In [7]:
Pkg.instantiate()

In [8]:
Pkg.add("CSV"); Pkg.add("DataFrames"); Pkg.add("LsqFit"); Pkg.add("Plots")

   Resolving package versions...
  No Changes to `D:\Documents\Documents Bertrand\_Poly\_H2022\MTH8408 Optimisation\Projet Optimisation git\Project.toml`
  No Changes to `D:\Documents\Documents Bertrand\_Poly\_H2022\MTH8408 Optimisation\Projet Optimisation git\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\Documents Bertrand\_Poly\_H2022\MTH8408 Optimisation\Projet Optimisation git\Project.toml`
  No Changes to `D:\Documents\Documents Bertrand\_Poly\_H2022\MTH8408 Optimisation\Projet Optimisation git\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\Documents Bertrand\_Poly\_H2022\MTH8408 Optimisation\Projet Optimisation git\Project.toml`
  No Changes to `D:\Documents\Documents Bertrand\_Poly\_H2022\MTH8408 Optimisation\Projet Optimisation git\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\Documents Bertrand\_Poly\_H2022\MTH8408 Optimisation\Projet Optimisation git\Project.toml`
  No Changes to `D

In [11]:
using DataFrames, LsqFit, CSV, Statistics, QuadraticModels, LinearAlgebra, SparseMatricesCOO

## Exploration des données

In [13]:
data_folder = "Data/"
file_name   = "weatherstats_montreal_daily.csv"
df = DataFrame(CSV.File(data_folder * file_name))
first(df, 5)

,date,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature
,Date,Float64,Float64,Float64,Float64
1,2022-03-31,10.1,4.85,5.3,0.5
2,2022-03-30,3.0,-1.26,-2.35,-7.7
3,2022-03-29,-0.6,-4.58,-4.0,-7.4
4,2022-03-28,-6.9,-9.75,-9.55,-12.2
5,2022-03-27,2.5,-1.65,-3.5,-9.5


In [14]:
"pop" in names(df)

false

In [15]:
nrow(df)

10000

In [16]:
ncol(df)

70

In [17]:
## Get the data
temp  = df.avg_temperature
jours = [i for i in 1:length(df.avg_temperature)]

10000-element Vector{Int64}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
  9989
  9990
  9991
  9992
  9993
  9994
  9995
  9996
  9997
  9998
  9999
 10000

## Régression non linéaire 

In [18]:
## Premier modèle classique du papier
# t: array of independent variable
# p: array of model parameters
modele_simple(t, p) = p[1] .+ p[2]*t .+ p[3]*cos.(2*pi*t/365.25) .+ p[4]*sin.(2*pi*t/365.25) .+  p[5]*cos.(2*pi*t/(10.7*365.25)) .+  p[6]*sin.(2*pi*t/(10.7*365.25))

modele_simple (generic function with 1 method)

In [19]:
## Modèle avec des sinusoide de période de 6 mois rajoutés comme amélioration du modèle
modele_harmonique(t, p) = p[1] .+ p[2]*t .+ p[3]*cos.(2*pi*t/365.25) .+ p[4]*sin.(2*pi*t/365.25) .+  p[5]*cos.(2*pi*t/(10.7*365.25)) .+  p[6]*sin.(2*pi*t/(10.7*365.25)) .+  p[7]*cos.(4*pi*t/(365.25)) .+  p[8]*sin.(4*pi*t/(365.25))

modele_harmonique (generic function with 1 method)

In [20]:
p0_S  = [2.6, 3.5, -20.0, -8.31, -0.197, 0.211]
fit_S = curve_fit(modele_simple, jours, temp, p0_S)

LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}}([7.692148549432855, -6.568308514819062e-5, -5.668829648232498, -14.250878022536483, 0.29210508841972127, 0.11832982248036909], [-3.2287506359537965, 4.178823772817111, 5.5882189895995875, 10.899506249032274, 4.6127562256460575, -3.3719609870498064, -4.75457589730963, -4.135019635715162, 0.4367760244447002, -0.47912135301560177  …  2.7411033400969025, -5.071473638767642, -7.25178536940131, -2.01989984383728, -7.685885704074471, -8.149812221764133, -3.21174927771005, -2.8717673411920273, -6.629937449111727, -8.036331184970487], [0.9999999999934213 0.9999999999502168 … 0.999998707677931 0.0016077024494147915; 0.9999999999934213 2.0000000000471077 … 0.9999948305677253 0.0032154007919543773; … ; 0.9999999999934213 9998.999999999956 … -0.9332426053829892 -0.35924676698286817; 0.9999999999934213 9999.99999999998 … -0.9326638372475743 -0.36074667913889624], true, Float64[])

In [21]:
p0_H  = [2.6, 3.5, -20.0, -8.31, -0.197, 0.211, 1.0, 2.0]
fit_H = curve_fit(modele_harmonique, jours, temp, p0_H)

LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}}([7.698429441854731, -6.624289856675415e-5, -5.663344240346553, -14.244929294302148, 0.29409049490138206, 0.11690742397564687, 0.4532455029624477, -0.7365648689491504], [-2.787259387923716, 4.594301254926464, 5.9772057761506145, 11.261556847931688, 4.947457108093387, -3.0649908986700476, -4.475684779969131, -3.8845223552478445, 0.6585982843864225, -0.28622128391554824  …  3.3336076927759946, -4.4577602963105045, -6.617589095877574, -1.3659710601130612, -7.012998305280007, -7.458762661583306, -2.503355625933702, -2.146868315650778, -5.88939142313369, -7.281015171538638], [0.9999999999938503 1.0000000000235538 … 0.9994082115903262 0.03439806059828307; 0.9999999999938503 1.9999999999004336 … 0.997633546806214 0.06875540849591907; … ; 0.9999999999938503 9998.999999999956 … 0.00967621241304924 -0.9999531843596766; 0.9999999999938503 9999.99999999998 … 0.04406693633545649 -0.9990285807367247], true, Float6

### Affichage des paramètres trouvés

In [22]:
fit_S.param

6-element Vector{Float64}:
   7.692148549432855
  -6.568308514819062e-5
  -5.668829648232498
 -14.250878022536483
   0.29210508841972127
   0.11832982248036909

In [23]:
fit_H.param

8-element Vector{Float64}:
   7.698429441854731
  -6.624289856675415e-5
  -5.663344240346553
 -14.244929294302148
   0.29409049490138206
   0.11690742397564687
   0.4532455029624477
  -0.7365648689491504

In [24]:
sum(fit_S.resid.^2)

213308.66581277817

In [25]:
sum(fit_H.resid.^2)

209578.65082257235

### Estimate goodness of fit

In [26]:
# se = standard_error(fit_S)
cov = estimate_covar(fit_S)

6×6 Matrix{Float64}:
  0.00866214   -1.29183e-6    2.85544e-5  …  -0.00037037  -0.000630293
 -1.29183e-6    2.58341e-10  -9.35249e-9      9.43813e-8   1.76955e-8
  2.85544e-5   -9.35249e-9    0.0042822       2.87306e-5   1.77622e-5
 -6.48364e-5    3.66064e-9   -1.1475e-5      -1.78494e-5   2.82712e-5
 -0.00037037    9.43813e-8    2.87306e-5      0.00422105  -5.32003e-5
 -0.000630293   1.76955e-8    1.77622e-5  …  -5.32003e-5   0.00449482

In [27]:
cov = estimate_covar(fit_H)

8×8 Matrix{Float64}:
  0.00851305   -1.26962e-6    2.87084e-5  …  -2.11797e-5  -4.88232e-5
 -1.26962e-6    2.53899e-10  -9.30871e-9      6.81852e-9   7.37583e-9
  2.87084e-5   -9.30871e-9    0.00420878     -2.31501e-5  -4.54935e-5
 -6.37979e-5    3.6435e-9    -1.13853e-5      3.90959e-5  -9.95886e-6
 -0.000363602   9.26779e-8    2.8589e-5      -1.90172e-5  -2.29909e-5
 -0.000619365   1.73762e-8    1.74917e-5  …  -1.02235e-5   1.84439e-6
 -2.11797e-5    6.81852e-9   -2.31501e-5      0.0041968   -1.17372e-5
 -4.88232e-5    7.37583e-9   -4.54935e-5     -1.17372e-5   0.00419495

## Plots

In [28]:
pred_S = modele_simple(jours, fit_S.param)
pred_H = modele_harmonique(jours, fit_H.param)

10000-element Vector{Float64}:
  2.512740612076284
  2.244301254926464
  1.9772057761506148
  1.7115568479316867
  1.447457108093387
  1.1850091013299524
  0.9243152200308691
  0.6654776447521553
  0.40859828438642254
  0.15377871608445176
 -0.09887987501854359
 -0.3492767562183422
 -0.5973117077449
  ⋮
 -1.0586684488683709
 -0.8133925854935192
 -0.5663923072240052
 -0.3177602963105046
 -0.06758909587757411
  0.1840289398869389
  0.43700169471999334
  0.691237338416695
  0.9466443740662978
  1.2031316843492221
  1.4606085768663102
  1.7189848284613616

In [29]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423


In [30]:
gr()
scatter(df.date, temp, label="Température de Montréal", markersize=1, markercolor="lightblue")
p = plot!(df.date, [pred_S pred_H], title="Prédiction des Modèles pour Montréal", label=["Modèle Simple" "Modèle avec Harmoniques"], linecolor=["red" "yellow"])
xlabel!("Time t")

## Modèle en norme L1

L'article initial argumentait qu'un modèle en norme L1, en utilisant la somme des valeurs absolus des erreurs plutôt que la somme de leurs carrés, performerait mieux dans cette situation. Comme les valeurs absolus font qu'on ne peut pas résoudre directement ce problème par optimisation linéaire, nous allons utiliser la même astuce que celle de l'article de rajouter des contraintes pour pouvoir écrire le problème d'une manière soluble par optimisation linéaire.

In [31]:
## Exemple de modèle quadratique de QuadraticsModels
Q = [6. 2. 1.
    2. 5. 2.
    1. 2. 4.]
c = [-8.; -3; -3]
A = [1. 0. 1.
    0. 2. 1.]
b = [0.; 3]
l = [0.;0;0]
u = [Inf; Inf; Inf]

3-element Vector{Float64}:
 Inf
 Inf
 Inf

In [ ]:
QM = QuadraticModel(c, SparseMatrixCOO(tril(Q)), A=SparseMatrixCOO(A), lcon=b, ucon=b,lvar=l, uvar=u, c0=0., name="QM")

In [35]:
SparseMatrixCOO(tril(Q))

3×3 SparseMatrixCOO{Float64, Int64} with 6 stored entries:
     ┌─────┐    
   1 │⠀⠄⠀⠀⠀│ > 0
   3 │⠀⠅⠨⠠⠀│ < 0
     └─────┘    
     ⠀1⠀⠀⠀3⠀    
     6 nonzeros    